<a href="https://colab.research.google.com/github/Aliasgharshinwari/MNIST-Based-Handwritten-Digit-Recognition-On-FPGA-using-Vivado-HLS/blob/main/brevitas_to_qonnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%pip install netron
%pip install brevitas
%pip install qonnx


In [8]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import brevitas.nn as qnn
from brevitas.core.quant import QuantType

# Define the Brevitas-based MNIST model
class QuantizedMNISTModel(nn.Module):
    def __init__(self):
        super(QuantizedMNISTModel, self).__init__()
        self.flatten = nn.Flatten()

        # First layer: Fully connected with quantized weights and activations
        self.fc1 = qnn.QuantLinear(
            in_features=28 * 28,
            out_features=128,
            weight_bit_width=8,
            bias=True,
            quant_type=QuantType.INT,
            return_quant_tensor=True
        )
        self.act1 = qnn.QuantReLU(activation_bit_width=8)

        # Second layer: Fully connected with quantized weights and activations
        self.fc2 = qnn.QuantLinear(
            in_features=128,
            out_features=64,
            weight_bit_width=8,
            bias=True,
            quant_type=QuantType.INT,
            return_quant_tensor=True
        )
        self.act2 = qnn.QuantReLU(activation_bit_width=8)

        # Output layer: Fully connected with quantized weights
        self.fc3 = qnn.QuantLinear(
            in_features=64,
            out_features=10,
            weight_bit_width=8,
            bias=True,
            quant_type=QuantType.INT,
            return_quant_tensor=True
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x

# Model initialization
model = QuantizedMNISTModel()

# Test with dummy data
dummy_input = torch.randn(1, 1, 28, 28)  # Batch size of 1, 1 channel, 28x28 image
output = model(dummy_input)
print("Output shape:", output.shape)


RuntimeError: QuantLayer is not correctly configured

In [ ]:
# Load MNIST dataset with data augmentation and normalization
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Check if GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss, and optimizer
model = QuantizedMNISTModel().to(device)  # Move model to GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        # Move data to the same device as the model (GPU if available)
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

# Test the model
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        # Move data to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


In [ ]:
%pip install onnx onnxoptimizer

In [ ]:
from brevitas.export import export_qonnx
import onnx
model = model.to('cuda')  # Move model to GPU
inp = torch.randn(1,1,28*28).to('cuda')
path = 'quant_model_qcdq.onnx'

quant_model = export_qonnx(model, inp, export_path=path)
onnx.save(quant_model, "path")

In [ ]:
%pip install netron

In [ ]:
import netron
from IPython.display import IFrame

# Path to the ONNX model
model_path = "quant_model_qcdq.onnx"

# Start the Netron server
port = 8081  # Choose an available port
url = netron.start(model_path, address=("localhost", port), browse=False)

# Display the visualization in an iframe
IFrame(src=f"http://{url}", width="100%", height="600px")


In [9]:
import torch

model = torch.load('quant_mobilenet_v1_4b-0100a667.pth')
print(model)

<ipython-input-9-de9f9fda55b0>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('quant_mobilenet_v1_4b-0100a667.pth')


OrderedDict([('features.init_block.conv.weight', tensor([[[[ 5.7093e-40,  1.2611e-40,  5.4048e-40],
          [ 6.2409e-40,  1.2262e-40,  5.0110e-40],
          [ 5.1726e-40,  2.4090e-40,  3.7996e-40]],

         [[ 6.2666e-40, -1.2611e-40,  5.8376e-40],
          [ 7.0458e-40, -1.9007e-40,  5.6312e-40],
          [ 5.2805e-40,  1.2611e-40,  3.8210e-40]],

         [[ 4.9464e-40,  1.2611e-40,  3.9930e-40],
          [ 5.2403e-40,  1.2611e-40,  4.1340e-40],
          [ 3.9069e-40,  2.2397e-40,  3.4976e-40]]],


        [[[ 2.2959e-01, -2.0918e-01, -1.0184e-02],
          [ 2.9467e-01, -1.4942e-01, -6.5507e-02],
          [ 1.8251e-01, -2.3527e-01, -4.5599e-03]],

         [[ 5.9259e-01, -1.8574e-01, -2.6957e-01],
          [ 6.2124e-01, -1.5427e-01, -3.5702e-01],
          [ 4.0211e-01, -2.8432e-01, -2.7638e-01]],

         [[ 1.9119e-01, -2.1344e-01, -5.4504e-02],
          [ 2.8509e-01, -1.1106e-01, -5.5353e-02],
          [ 1.4585e-01, -1.6867e-01,  4.6168e-02]]],


        [[[ 2.419